## Зачем собирать данные автоматически? 

<br>

<br>

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/aaaaaa.png" width="500"> 

## Что такое HTML? 

**HTML (HyperText Markup Language)**  — это такой же язык разметки как Markdown или LaTeX. Он является стандартным для написания различных сайтов. Команды в таком языке называются **тегами**. Если открыть абсолютно любой сайт, нажать на правую кнопку мышки, а после нажать `View page source`, то перед вами предстанет HTML скелет этого сайта. 

Вот так можно открыть html-код стартовой яндекса, найти в нём новость из топа и немного её изменить. Понятное дело, что вы так меняете html-страничку только на своём компьютере (клиенте). На сервере яндекса остаётся исходная версия. Когда вы обновите страничку все правки исчезнут. Однако никто вам не запрещает ворваться вк в какую-нибудь переписку, открыть её код, поменять текст нескольких сообщений и наделать скринов для мемов. 

<br> 
<img src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/gifs/yandex.gif" width="700"> 
<br> 


HTML-страница это ни что иное, как набор вложенных тегов. Можно заметить, например, следующие теги:

- `<title>` – заголовок страницы
- `<h1>…<h6>` – заголовки разных уровней
- `<p>` – абзац (paragraph)
- `<div>` – выделения фрагмента документа с целью изменения вида содержимого
- `<table>` – прорисовка таблицы 
- `<tr>` – разделитель для строк в таблице 
- `<td>` – разделитель для столбцов в таблице
- `<b>` – устанавливает жирное начертание шрифта

Обычно команда `<...>` открывает тег, а  `</...>` закрывает его. Все, что находится между этими двумя командами, подчиняется правилу, которое диктует тег. Например, все, что находится между `<p>` и  `</p>` — это отдельный абзац.   

Теги образуют своеобразное дерево с корнем в теге `<html>` и разбивают страницу на разные логические кусочки. У каждого тега есть свои потомки (дети) — те теги, которые вложены в него и свои родители. 

Например, HTML-древо страницы может выглядеть вот так:


````
<html>
<head> Заголовок </head>
<body>
    <div>
        Первый кусок текста со своими свойствами
    </div>
    <div>
        Второй кусок текста
            <b>
                Третий, жирный кусок
            </b>
    </div>
    Четвёртый кусок текста
</body>
</html>
````

Можно работать с этим html как с текстом, а можно как с деревом. Обход этого дерева и есть парсинг веб-страницы. Мы всего лишь будем находить нужные нам узлы среди всего этого разнообразия и забирать из них информацию!

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/tree.png" width="450"> 

# Скачиваем табличку с сайта ЦБ


Доступ к веб-станицам позволяет получать модуль requests. Подгрузим его. Если у вас не установлен этот модуль, то нужно установить:  `pip install requests`.



In [ ]:
!pip install requests

In [ ]:
import requests

In [ ]:
resp = requests.get('https://cbr.ru/currency_base/daily/')


Внутри response лежит html-разметка странички, которую мы парсим. 

In [ ]:
resp

<Response [200]>

In [ ]:
resp.content[:100]

b'\r\n<!DOCTYPE html>\r\n<html>\r\n<head>\r\n    \r\n\r\n<meta http-equiv="Content-Type" content="text/html; chars'

Выглядит неудобоваримо, как насчет сварить из этого дела что-то покрасивее? Например, прекрасный суп.

<img align="center" src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/alisa.jpg" height="200" width="200"> 

Пакет **[`bs4`](https://www.crummy.com/software/BeautifulSoup/)**, a.k.a **BeautifulSoup** был назван в честь стишка про красивый суп из Алисы в стране чудес. Эта совершенно волшебная библиотека, которая из сырого и необработанного HTML (или XML) кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что вам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Так что нам нужен пакет `beautifulsoup4`. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — `bs4`, а импортировать функцию под названием `BeautifulSoup`. 

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
tree = BeautifulSoup(resp.content, 'html.parser')
tree

В коде страницы мы можем найти где именно для каждого курса лежит основная информация. Видно, что она находится внутри тега `table`, для которого прописан класс `data` (грубо говоря, в html класс задаёт оформление соотвествующего кусочка страницы). 


Вытащим информацию о курсе валюты 

In [ ]:
# нашли табличку
table = tree.find_all('table', {'class' : 'data'})[0]
table

<table class="data">
<tbody>
<tr>
<th>Цифр. код</th>
<th>Букв. код</th>
<th>Единиц</th>
<th>Валюта</th>
<th>Курс</th>
</tr>
<tr>
<td>036</td>
<td>AUD</td>
<td>1</td>
<td>Австралийский доллар</td>
<td>54,0167</td>
</tr>
<tr>
<td>944</td>
<td>AZN</td>
<td>1</td>
<td>Азербайджанский манат</td>
<td>43,2267</td>
</tr>
<tr>
<td>051</td>
<td>AMD</td>
<td>100</td>
<td>Армянских драмов</td>
<td>14,8985</td>
</tr>
<tr>
<td>933</td>
<td>BYN</td>
<td>1</td>
<td>Белорусский рубль</td>
<td>29,1495</td>
</tr>
<tr>
<td>975</td>
<td>BGN</td>
<td>1</td>
<td>Болгарский лев</td>
<td>44,3866</td>
</tr>
<tr>
<td>986</td>
<td>BRL</td>
<td>1</td>
<td>Бразильский реал</td>
<td>14,2038</td>
</tr>
<tr>
<td>348</td>
<td>HUF</td>
<td>100</td>
<td>Венгерских форинтов</td>
<td>24,8628</td>
</tr>
<tr>
<td>410</td>
<td>KRW</td>
<td>1000</td>
<td>Вон Республики Корея</td>
<td>62,8933</td>
</tr>
<tr>
<td>344</td>
<td>HKD</td>
<td>10</td>
<td>Гонконгских долларов</td>
<td>94,4338</td>
</tr>
<tr>
<td>208</td>
<td>DKK</td>

***Задание: позже вернуться к этой задаче и формить результат запроса в виде таблицы Pandas ***

## Качаем цены на книги

* Хотим собрать [цены на книги](http://books.toscrape.com)
* Руками долго, напишем код на питоне


In [ ]:
import requests  

response = requests.get('http://books.toscrape.com/catalogue/page-1.html')
response

<Response [200]>

Благословенный 200 ответ - соединение установлено и данные получены, всё чудесно! Если попытаться перейти на несуществующую страницу, то можно получить, например, знаменитую ошибку 404.

In [ ]:
requests.get('http://books.toscrape.com/big_scholarship')

<Response [404]>

Внутри response лежит html-разметка странички, которую мы парсим. 

In [ ]:
response.content[:1000]

b'\n\n<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:30" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon"

In [ ]:
from bs4 import BeautifulSoup

# распарсили страничку в дерево 
tree = BeautifulSoup(response.content, 'html.parser')
tree


<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:30" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="../static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="../static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link 

Внутри переменной `tree` теперь лежит дерево из тегов, по которому мы можем совершенно спокойно бродить. 

In [ ]:
tree.html.head.title

<title>
    All products | Books to Scrape - Sandbox
</title>

Более того, зная адрес элемента, мы сразу можем найти его. Например, вот так в коде страницы мы можем найти где именно для каждой книги лежит основная информация. Видно, что она находится внутри тега `article`, для которого прописан класс `product_pod` (грубо говоря, в html класс задаёт оформление соотвествующего кусочка страницы). 


Вытащим информацию о книге из этого тега. 

In [ ]:
books = tree.find_all('article', {'class' : 'product_pod'})
books[0]

<article class="product_pod">
<div class="image_container">
<a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём.

In [ ]:
type(books[0])

bs4.element.Tag

In [ ]:
books[0].find('p', {'class': 'price_color'}).text

'£51.77'

Обратите внимание, что для поиска есть как минимум два метода: `find` и `find_all`. Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый. Чтобы найти все элементы с таким адресом, нужно использовать метод `find_all`. На выход будет выдан список.

Кроме содержимого у тегов часто есть атрибуты. Например, у названия книги есть атрибуты `title` и `href`: 

In [ ]:
books[0].h3

<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>

Их тоже можно вытащить.

In [ ]:
books[0].h3.a.get('href')

'a-light-in-the-attic_1000/index.html'

In [ ]:
books[0].h3.a.get('title')

'A Light in the Attic'

А ещё по этим атрибутам можно искать интересующие нас кусочки страницы. 

In [ ]:
tree.find_all('a', {'title': 'A Light in the Attic'})

[<a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>]



Обратите внимание, что на сайте все книги лежат на разных страничках. Если попробовать потыкать их, можно заметить, что в ссылке будет меняться атрибут `page`. Значит, если мы хотим собрать все книги, надо создать кучу ссылок с разным `page` внутри цикла. 


Давайте запишем весь код для сбора книг в виде функции. На вход она будет принимать номер странички, которую надо скачать. 

In [ ]:

infa = [ ]
    
for book in books:
       infa.append({'price': book.find('p', {'class': 'price_color'}).text,
                  
                    'title': book.h3.a.get('title')})

In [ ]:
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
r = requests.get('https://www.amazon.com', headers)
r.status_code

503

In [ ]:
infa

[{'href': 'a-light-in-the-attic_1000/index.html',
  'price': '£51.77',
  'title': 'A Light in the Attic'},
 {'href': 'tipping-the-velvet_999/index.html',
  'price': '£53.74',
  'title': 'Tipping the Velvet'},
 {'href': 'soumission_998/index.html',
  'price': '£50.10',
  'title': 'Soumission'},
 {'href': 'sharp-objects_997/index.html',
  'price': '£47.82',
  'title': 'Sharp Objects'},
 {'href': 'sapiens-a-brief-history-of-humankind_996/index.html',
  'price': '£54.23',
  'title': 'Sapiens: A Brief History of Humankind'},
 {'href': 'the-requiem-red_995/index.html',
  'price': '£22.65',
  'title': 'The Requiem Red'},
 {'href': 'the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
  'price': '£33.34',
  'title': 'The Dirty Little Secrets of Getting Your Dream Job'},
 {'href': 'the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
  'price': '£17.93',
  'title': 'The Coming Woman: A Novel Based on the Life of the Infamous Femin

In [ ]:
import pandas as pd

df = pd.DataFrame(infa)
print(df.shape)
df.head()

(20, 2)


,price,title
0,£51.77,A Light in the Attic
1,£53.74,Tipping the Velvet
2,£50.10,Soumission
3,£47.82,Sharp Objects
4,£54.23,Sapiens: A Brief History of Humankind


In [ ]:
def get_page(p):
    
    # изготовили ссылку
    url = 'http://books.toscrape.com/catalogue/page-{}.html'.format(p)
    
    # сходили по ней
    response = requests.get(url)
    
    # построили дерево 
    tree = BeautifulSoup(response.content, 'html.parser')
    
    # нашли в нём всё самое интересное
    books = tree.find_all('article', {'class' : 'product_pod'})
    
    infa = [ ]
    
    for book in books:
        infa.append({'price': book.find('p', {'class': 'price_color'}).text,
                     'href': book.h3.a.get('href'),
                     'title': book.h3.a.get('title')})
                     
    return infa

Осталось только пройтись по всем страничкам от page-1 до page-50 циклом и данные у нас в кармане. 

In [ ]:
infa = []

for p in range(1,51):
    infa.extend(get_page(p))

In [ ]:
import pandas as pd

df = pd.DataFrame(infa)
print(df.shape)
df.head()

(1000, 3)


,price,href,title
0,£51.77,a-light-in-the-attic_1000/index.html,A Light in the Attic
1,£53.74,tipping-the-velvet_999/index.html,Tipping the Velvet
2,£50.10,soumission_998/index.html,Soumission
3,£47.82,sharp-objects_997/index.html,Sharp Objects
4,£54.23,sapiens-a-brief-history-of-humankind_996/index...,Sapiens: A Brief History of Humankind


Кстати говоря, если перейти по ссылке в саму книгу, там о ней будет куча дополнительной информации. Можно пройтись по всем ссылкам и выкачать себе по ним дополнительную информацию. 

In [ ]:
df.to_excel('books.xls')

In [ ]:
from google.colab import files
files.download('books.xls')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>